In [2]:
!nvidia-smi

Fri Jan 13 11:44:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install transformers

In [4]:
import warnings
warnings.simplefilter('ignore')

import os
import time
import copy
import random

import numpy as np
import pandas as pd
pd.set_option('max_columns', None)
pd.set_option('max_colwidth', 128)
from tqdm.notebook import tqdm

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup, AutoConfig

# 参数设置

class Config:
    def __init__(self):
        super(Config, self).__init__()

        self.SEED = 71
        self.MODEL_PATH = 'roberta-large'
        self.NUM_LABELS = 20

        # data
        self.TOKENIZER = AutoTokenizer.from_pretrained(self.MODEL_PATH)
        self.MAX_LENGTH = 128
        self.BATCH_SIZE = 16
        self.VALIDATION_SPLIT = 0.10

        # model
        self.DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.FULL_FINETUNING = True
        self.LR = 3e-5
        self.OPTIMIZER = 'AdamW'
        self.N_VALIDATE_DUR_TRAIN = 3
        self.N_WARMUP = 0
        self.SAVE_BEST_ONLY = True
        self.EPOCHS = 15
        self.USE_FGM = False


config = Config()


def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


np.random.seed(config.SEED)
seed_torch(seed=config.SEED)

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [5]:
datadir = "/kaggle/input/valueaaa"

In [6]:
import numpy as np
import pandas as pd
def argumentadd(filepathArg,filepathLal):
    trainArg = pd.read_csv(filepathArg, sep='\t', header=0)
    trainLal = pd.read_csv(filepathLal, sep='\t', header=0)
    def dataframetoList(X):
        train_data = np.array(X)#np.ndarray()
        train_x_list=train_data.tolist()#list
        return train_x_list
    
    trainArgList = dataframetoList(trainArg)
    
    def insertSentence(dataList):
        for x in range(0, len(dataList)):
            sentence = "Imagine someone is arguing " + str(dataList[x][2]) + " \"" + str(dataList[x][1]) + "\" " + "by saying: \"" + str(dataList[x][3]) + "\"."
            dataList[x].append(sentence)
        return dataList
    trainArgs = insertSentence(trainArgList)
    
    # list 转dataframe
    trainArgD = pd.DataFrame(trainArgs)
    trainArgD.columns = ["Argument ID","Conclusion","Stance","Premise","sentence"]
    
    trainArgD.drop(columns=["Conclusion","Stance","Premise"],inplace=True)
    train_merge = trainArgD.merge(trainLal,on='Argument ID')
    
    return train_merge

In [53]:
train_df = argumentadd(datadir+'/arguments-training.tsv',datadir + '/labels-training.tsv')
valid_df = argumentadd(datadir+'/arguments-validation.tsv',datadir + '/labels-validation.tsv')

In [8]:
def testcreate(filepathArg):
    test = pd.read_csv(filepathArg, sep='\t', header=0)
    def dataframetoList(X):
        train_data = np.array(X)#np.ndarray()
        train_x_list=train_data.tolist()#list
        return train_x_list
    def insertSentence(dataList):
        for x in range(0, len(dataList)):
            sentence = "Imagine someone is arguing " + str(dataList[x][2]) + " \"" + str(dataList[x][1]) + "\" " + "by saying: \"" + str(dataList[x][3]) + "\"."
            dataList[x].append(sentence)
        return dataList
    
    testList = dataframetoList(test)
    testLists = insertSentence(testList)
    testD = pd.DataFrame(testLists)
    testD.columns = ["Argument ID","Conclusion","Stance","Premise","sentence"]
    testD.drop(columns=["Conclusion","Stance","Premise"],inplace=True)
    return testD

In [9]:
test_df = testcreate(datadir+'/arguments-test.tsv')
test2_df = testcreate(datadir+'/arguments-test-nahjalbalagha.tsv')

In [10]:
class TransformerDataset(Dataset):
    def __init__(self, df, set_type=None):
        super(TransformerDataset, self).__init__()

        #df = df.iloc[indices]
        self.texts = df['sentence'].values.tolist()
        self.set_type = set_type
        if self.set_type != 'test':
            self.labels = df.iloc[:, 2:].values

        self.tokenizer = config.TOKENIZER
        self.max_length = config.MAX_LENGTH

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        tokenized = self.tokenizer.encode_plus(
            self.texts[index],
            add_special_tokens=True,
            max_length=self.max_length,
            pad_to_max_length=True,
            truncation=True,
            return_attention_mask=True,
            return_token_type_ids=False,
            return_tensors='pt'
        )
        input_ids = tokenized['input_ids'].squeeze()
        attention_mask = tokenized['attention_mask'].squeeze()

        if self.set_type != 'test':
            return {
                'input_ids': input_ids.long(),
                'attention_mask': attention_mask.long(),
                'labels': torch.Tensor(self.labels[index]).float(),
            }

        return {
            'input_ids': input_ids.long(),
            'attention_mask': attention_mask.long(),
        }

In [11]:
train_data = TransformerDataset(train_df)
valid_data = TransformerDataset(valid_df)

train_dataloader = DataLoader(train_data, batch_size=config.BATCH_SIZE, shuffle=True)
valid_dataloader = DataLoader(valid_data, batch_size=config.BATCH_SIZE, shuffle=False)

In [12]:
class FGM(object):
    def __init__(self, model, emb_name, epsilon=1.0):
        self.model = model
        self.epsilon = epsilon
        self.emb_name = emb_name
        self.backup = {}

    def attack(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad and self.emb_name in name:
                self.backup[name] = param.data.clone()
                norm = torch.norm(param.grad)
                if norm != 0 and not torch.isnan(norm):
                    r_at = self.epsilon * param.grad / norm
                    param.data.add_(r_at)

    def restore(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad and self.emb_name in name:
                assert name in self.backup
                param.data = self.backup[name]
        self.backup = {}

In [13]:
def init_params(module_lst):
    for module in module_lst:
        for param in module.parameters():
            if param.dim() > 1:
                torch.nn.init.xavier_uniform_(param)
    return

In [21]:
class Model(nn.Module):
    def __init__(self, ):
        super().__init__()

        cfg = AutoConfig.from_pretrained(config.MODEL_PATH)
        cfg.update({"output_hidden_states": True,
                    "hidden_dropout_prob": 0.0,
                    "layer_norm_eps": 1e-7})

        self.roberta = AutoModel.from_pretrained(config.MODEL_PATH, config=cfg)

        dim = self.roberta.pooler.dense.bias.shape[0]

        self.dropout = nn.Dropout(p=0.2)
        self.high_dropout = nn.Dropout(p=0.5)

        n_weights = 24
        weights_init = torch.zeros(n_weights).float()
        weights_init.data[:-1] = -3
        self.layer_weights = torch.nn.Parameter(weights_init)

        self.attention = nn.Sequential(
            nn.Linear(1024, 1024),
            nn.Tanh(),
            nn.Linear(1024, 1),
            nn.Softmax(dim=1)
        )
        self.cls = nn.Sequential(
            nn.Linear(dim, 20)
        )
        init_params([self.cls, self.attention])

    def forward(self, input_ids, attention_mask):
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)

        cls_outputs = torch.stack(
            [self.dropout(layer) for layer in roberta_output[2][-24:]], dim=0
        )
        # print("处理前：",cls_outputs.shape)
        cls_output = (
                torch.softmax(self.layer_weights, dim=0).unsqueeze(1).unsqueeze(1).unsqueeze(1) * cls_outputs).sum(
            0)
        # print("处理后：",cls_outputs.shape)
        # print(type(cls_outputs))
        logits = torch.mean(
            torch.stack(
                [torch.sum(self.attention(self.high_dropout(cls_output)) * cls_output, dim=1) for _ in range(5)],
                dim=0,
            ),
            dim=0,
        )
        #print("logits:",logits.shape)
        return self.cls(logits)
    
device = config.DEVICE

In [15]:
pip install torchmetrics

Note: you may need to restart the kernel to use updated packages.


In [16]:
from torchmetrics.classification import MultilabelF1Score
metric = MultilabelF1Score(num_labels=20,average="macro").to(device)

In [17]:
import torch.nn.functional as F
def val(model, valid_dataloader, criterion, metric):
    val_loss = 0
    true, pred = [], []

    # set model.eval() every time during evaluation
    model.eval()
    metric.reset()

    for step, batch in enumerate(valid_dataloader):
        b_input_ids = batch['input_ids'].to(device)
        b_attention_mask = batch['attention_mask'].to(device)
        b_labels = batch['labels'].to(device)

        with torch.no_grad():
            # forward pass
            logits = model(input_ids=b_input_ids, attention_mask=b_attention_mask)
            probs = F.sigmoid(logits)
            metric.update(probs, b_labels)


            # calculate loss
            loss = criterion(logits, b_labels)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(valid_dataloader)
    f1_score= float(metric.compute())
    print("eval loss: %.5f, f1 score: %.5f" %
          (avg_val_loss, f1_score))
    return f1_score

In [18]:
def run():
    # setting a seed ensures reproducible results.
    # seed may affect the performance too.
    torch.manual_seed(config.SEED)

    criterion = nn.BCEWithLogitsLoss()
    #criterion = RMSELoss()

    # define the parameters to be optmized -
    # - and add regularization
    if config.FULL_FINETUNING:
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        optimizer_parameters = [
            {
                "params": [
                    p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
                ],
                "weight_decay": 0.001,
            },
            {
                "params": [
                    p for n, p in param_optimizer if any(nd in n for nd in no_decay)
                ],
                "weight_decay": 0.0,
            },
        ]
        optimizer = optim.AdamW(optimizer_parameters, lr=config.LR)

    num_training_steps = len(train_dataloader) * config.EPOCHS
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    max_f1_score = 0
    for epoch in range(config.EPOCHS):
        #best_model, best_val_mse_score = train(model, train_dataloader, valid_dataloader, criterion, optimizer, scheduler, epoch, metric)
        f1_score = val(model, valid_dataloader, criterion, metric)
        if config.USE_FGM:
          fgm = FGM(model, epsilon=1, emb_name='word_embeddings.')

        #train_loss = 0
        for step, batch in enumerate(tqdm(train_dataloader,
                                          desc='Epoch ' + str(epoch))):
            # set model.eval() every time during training
            model.train()

            # unpack the batch contents and push them to the device (cuda or cpu).
            b_input_ids = batch['input_ids'].to(device)
            b_attention_mask = batch['attention_mask'].to(device)
            b_labels = batch['labels'].to(device)

            # clear accumulated gradients
            optimizer.zero_grad()

            # forward pass
            logits = model(input_ids=b_input_ids, attention_mask=b_attention_mask)
            probs = F.sigmoid(logits)
            metric.update(probs, b_labels)
            f1_score= float(metric.compute())

            # calculate loss
            loss = criterion(logits, b_labels)
            #train_loss += loss.item()
            if step % 10 == 0:
                print(
                    "step %d, loss: %.5f, f1 score: %.5f"
                    % (step, loss, f1_score,
                        ))

            # backward pass
            loss.backward()
            
            # fgm attack
            if config.USE_FGM:
                fgm.attack()
                logits_adv = model(input_ids=b_input_ids, attention_mask=b_attention_mask)
                loss_adv = criterion(logits_adv, b_labels)
                loss_adv.backward()
                fgm.restore()

            # update weights
            optimizer.step()

            # update scheduler
            scheduler.step()

            mertic1 = MultilabelF1Score(num_labels=20,average="macro").to(device)
            if step % 40 == 0:
                print(f'-- Step: {step}')
                f1_score = val(model, valid_dataloader, criterion, mertic1)

                # min_f1_score = float('inf')
                if config.SAVE_BEST_ONLY:
                    if f1_score > max_f1_score:
                        best_model = copy.deepcopy(model)
                        best_val_mse_score = f1_score

                        model_name = 'roberta_best_model'
                        torch.save(best_model.state_dict(), model_name + '.pt')

                        print(f'--- Best Model. Val loss: {max_f1_score} -> {f1_score}')
                        max_f1_score = f1_score

        #avg_train_loss = train_loss / len(train_dataloader)
        #print('Training loss:', avg_train_loss)
        #print("Training loss:: %.5f, f1 score: %.5f" %
              #(avg_train_loss, f1_score))


    return best_model, best_val_mse_score

In [22]:
model = Model()
model.to(device)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm

In [23]:
best_model, best_F1score = run()

eval loss: 0.89020, f1 score: 0.15596


Epoch 0:   0%|          | 0/338 [00:00<?, ?it/s]

step 0, loss: 0.91733, f1 score: 0.15609
-- Step: 0
eval loss: 0.71285, f1 score: 0.14105
--- Best Model. Val loss: 0 -> 0.14104756712913513
step 10, loss: 0.41007, f1 score: 0.15518
step 20, loss: 0.41795, f1 score: 0.15276
step 30, loss: 0.38134, f1 score: 0.15121
step 40, loss: 0.41716, f1 score: 0.14921
-- Step: 40
eval loss: 0.41627, f1 score: 0.01854
step 50, loss: 0.46347, f1 score: 0.14785
step 60, loss: 0.40921, f1 score: 0.14620
step 70, loss: 0.37673, f1 score: 0.14804
step 80, loss: 0.42127, f1 score: 0.14877
-- Step: 80
eval loss: 0.40277, f1 score: 0.10422
step 90, loss: 0.32208, f1 score: 0.15121
step 100, loss: 0.32840, f1 score: 0.15243
step 110, loss: 0.40268, f1 score: 0.15453
step 120, loss: 0.34662, f1 score: 0.15666
-- Step: 120
eval loss: 0.37806, f1 score: 0.14914
--- Best Model. Val loss: 0.14104756712913513 -> 0.14914363622665405
step 130, loss: 0.37443, f1 score: 0.16002
step 140, loss: 0.33630, f1 score: 0.16783
step 150, loss: 0.37870, f1 score: 0.17319
ste

Epoch 1:   0%|          | 0/338 [00:00<?, ?it/s]

step 0, loss: 0.29046, f1 score: 0.35843
-- Step: 0
eval loss: 0.32084, f1 score: 0.36175
--- Best Model. Val loss: 0.34748539328575134 -> 0.36175423860549927
step 10, loss: 0.24900, f1 score: 0.37386
step 20, loss: 0.25318, f1 score: 0.39078
step 30, loss: 0.31662, f1 score: 0.40585
step 40, loss: 0.33044, f1 score: 0.41554
-- Step: 40
eval loss: 0.32422, f1 score: 0.37590
--- Best Model. Val loss: 0.36175423860549927 -> 0.3759002089500427
step 50, loss: 0.25570, f1 score: 0.42169
step 60, loss: 0.29474, f1 score: 0.42810
step 70, loss: 0.25612, f1 score: 0.43196
step 80, loss: 0.35485, f1 score: 0.43786
-- Step: 80
eval loss: 0.32439, f1 score: 0.35122
step 90, loss: 0.20908, f1 score: 0.44406
step 100, loss: 0.27556, f1 score: 0.44705
step 110, loss: 0.25028, f1 score: 0.44998
step 120, loss: 0.33831, f1 score: 0.45392
-- Step: 120
eval loss: 0.32188, f1 score: 0.39759
--- Best Model. Val loss: 0.3759002089500427 -> 0.39759236574172974
step 130, loss: 0.32564, f1 score: 0.45792
step

...

step 0, loss: 0.00430, f1 score: 0.45896
-- Step: 0
eval loss: 0.50600, f1 score: 0.45273
step 10, loss: 0.01343, f1 score: 0.51384
step 20, loss: 0.00402, f1 score: 0.56518
step 30, loss: 0.01598, f1 score: 0.60877
step 40, loss: 0.00437, f1 score: 0.63893
-- Step: 40
eval loss: 0.50801, f1 score: 0.45432
step 50, loss: 0.00363, f1 score: 0.66278
step 60, loss: 0.01060, f1 score: 0.68471
step 70, loss: 0.00339, f1 score: 0.70286
step 80, loss: 0.02923, f1 score: 0.72045
-- Step: 80
eval loss: 0.51103, f1 score: 0.44938
step 90, loss: 0.00472, f1 score: 0.73584
step 100, loss: 0.00334, f1 score: 0.74901
step 110, loss: 0.00444, f1 score: 0.76063
step 120, loss: 0.00423, f1 score: 0.77122
-- Step: 120
eval loss: 0.51380, f1 score: 0.45143
step 130, loss: 0.00458, f1 score: 0.78151
step 140, loss: 0.00327, f1 score: 0.79141
step 150, loss: 0.00591, f1 score: 0.79936
step 160, loss: 0.01181, f1 score: 0.80657
-- Step: 160
eval loss: 0.51400, f1 score: 0.44963
step 170, loss: 0.02525, f1 s

Epoch 13:   0%|          | 0/338 [00:00<?, ?it/s]

step 0, loss: 0.00335, f1 score: 0.45582
-- Step: 0
eval loss: 0.51422, f1 score: 0.44935
step 10, loss: 0.00465, f1 score: 0.51202
step 20, loss: 0.00255, f1 score: 0.55619
step 30, loss: 0.01149, f1 score: 0.59103
step 40, loss: 0.00383, f1 score: 0.62576
-- Step: 40
eval loss: 0.51559, f1 score: 0.45256
step 50, loss: 0.00375, f1 score: 0.64945
step 60, loss: 0.00315, f1 score: 0.67345
step 70, loss: 0.00348, f1 score: 0.69350
step 80, loss: 0.00336, f1 score: 0.71256
-- Step: 80
eval loss: 0.51857, f1 score: 0.44788
step 90, loss: 0.00263, f1 score: 0.72645
step 100, loss: 0.00388, f1 score: 0.74103
step 110, loss: 0.00370, f1 score: 0.75506
step 120, loss: 0.00316, f1 score: 0.76635
-- Step: 120
eval loss: 0.51728, f1 score: 0.45272
step 130, loss: 0.00330, f1 score: 0.77757
step 140, loss: 0.01207, f1 score: 0.78603
step 150, loss: 0.00323, f1 score: 0.79426
step 160, loss: 0.00393, f1 score: 0.80245
-- Step: 160
eval loss: 0.51893, f1 score: 0.45086
step 170, loss: 0.01649, f1 s

Epoch 14:   0%|          | 0/338 [00:00<?, ?it/s]

step 0, loss: 0.00289, f1 score: 0.45382
-- Step: 0
eval loss: 0.51939, f1 score: 0.44806
step 10, loss: 0.01726, f1 score: 0.51464
step 20, loss: 0.01572, f1 score: 0.56026
step 30, loss: 0.01732, f1 score: 0.60136
step 40, loss: 0.00336, f1 score: 0.63118
-- Step: 40
eval loss: 0.52014, f1 score: 0.45002
step 50, loss: 0.00319, f1 score: 0.65590
step 60, loss: 0.00316, f1 score: 0.68075
step 70, loss: 0.00366, f1 score: 0.69999
step 80, loss: 0.00359, f1 score: 0.71679
-- Step: 80
eval loss: 0.52146, f1 score: 0.44938
step 90, loss: 0.00324, f1 score: 0.73181
step 100, loss: 0.00385, f1 score: 0.74469
step 110, loss: 0.00366, f1 score: 0.75706
step 120, loss: 0.01387, f1 score: 0.76720
-- Step: 120
eval loss: 0.52111, f1 score: 0.44843
step 130, loss: 0.00416, f1 score: 0.77674
step 140, loss: 0.00352, f1 score: 0.78603
step 150, loss: 0.00337, f1 score: 0.79410
step 160, loss: 0.00390, f1 score: 0.80213
-- Step: 160
eval loss: 0.52108, f1 score: 0.45057
step 170, loss: 0.01142, f1 s

In [24]:
best_model

Model(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm

In [25]:
test_data = TransformerDataset(test_df, set_type='test')
test_dataloader = DataLoader(test_data, batch_size=config.BATCH_SIZE)
test2_data = TransformerDataset(test2_df, set_type='test')
test2_dataloader = DataLoader(test2_data, batch_size=config.BATCH_SIZE)

In [26]:
def predict(model,test_dataloader):
    val_loss = 0
    test_pred = []
    model.eval()
    for step, batch in enumerate(test_dataloader):
        b_input_ids = batch['input_ids'].to(device)
        b_attention_mask = batch['attention_mask'].to(device)

        with torch.no_grad():
            logits = model(input_ids=b_input_ids, attention_mask=b_attention_mask)
            logits = logits.cpu().numpy()
            test_pred.extend(logits)

    test_pred = np.array(test_pred)
    return test_pred

In [27]:
test_pred = predict(best_model,test_dataloader)
test2_pred = predict(best_model,test2_dataloader)

In [28]:
test_probs = F.sigmoid(torch.tensor(test_pred))
test2_probs = F.sigmoid(torch.tensor(test2_pred))

In [29]:
label_vocab = ["Self-direction: thought","Self-direction: action","Stimulation","Hedonism","Achievement","Power: dominance","Power: resources","Face","Security: personal","Security: societal","Tradition","Conformity: rules","Conformity: interpersonal","Humility","Benevolence: caring","Benevolence: dependability","Universalism: concern","Universalism: nature","Universalism: tolerance","Universalism: objectivity"]

In [30]:
def computeResult(test_probs):
    results = []
    for prob in test_probs:
            result = []
            for c, pred in enumerate(prob):
                if pred > 0.5:
                    result.append(label_vocab[c])
            results.append(','.join(result))
    return results

In [31]:
results = computeResult(test_probs)
results2 = computeResult(test2_probs)

In [32]:
def testOutput(results2,test_df):
    testData = test_df
    dicttestT =testData.to_dict("list")

    testPred = {}
    testPred["Argument ID"] = dicttestT["Argument ID"]
    testPred["sentence"] = dicttestT["sentence"]

    for x in label_vocab:
        testPred[x] = []

    for x in range(len(results2)):
        types = results2[x].split(",")
        if types == ['']:
            for y in label_vocab:
                testPred[y].append(0)
        else:  
            for z in label_vocab:
                if z in types:
                    testPred[z].append(1)
                else:
                    testPred[z].append(0)
    testPredD = pd.DataFrame.from_dict(testPred)
    testPredD.drop(columns=["sentence"],inplace=True)

    return testPredD

In [33]:
test = testOutput(results,test_df)
test.to_csv('/kaggle/working/testRoberta.tsv',columns=test.columns.tolist(),
            sep='\t',
            index=False)

In [34]:
test2 = testOutput(results2,test2_df)
test2.to_csv('/kaggle/working/test222Roberta.tsv',columns=test2.columns.tolist(),
            sep='\t',
            index=False)

In [35]:
valid_pred = predict(best_model,valid_dataloader)

In [45]:
valid_probs = F.sigmoid(torch.tensor(valid_pred))
validresults = computeResult(valid_probs)

In [54]:
valid_df

,Argument ID,sentence,Self-direction: thought,Self-direction: action,Stimulation,Hedonism,Achievement,Power: dominance,Power: resources,Face,Security: personal,Security: societal,Tradition,Conformity: rules,Conformity: interpersonal,Humility,Benevolence: caring,Benevolence: dependability,Universalism: concern,Universalism: nature,Universalism: tolerance,Universalism: objectivity
0,A01001,"Imagine someone is arguing in favor of ""Entrapment should be legalized"" by saying: ""if entrapment can serve to more easily c...",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,A01012,"Imagine someone is arguing in favor of ""The use of public defenders should be mandatory"" by saying: ""the use of public defen...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,A02001,"Imagine someone is arguing in favor of ""Payday loans should be banned"" by saying: ""payday loans create a more impoverished s...",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
3,A02002,"Imagine someone is arguing against ""Surrogacy should be banned"" by saying: ""Surrogacy should not be banned as it is the woma...",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,A02009,"Imagine someone is arguing against ""Entrapment should be legalized"" by saying: ""entrapment is gravely immoral and against hu...",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1891,E08014,"Imagine someone is arguing in favor of ""We should shift the EU policy toward the Russian Federation"" by saying: ""Pushing Rus...",1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1
1892,E08021,"Imagine someone is arguing in favor of ""We should stop buying Russian gas"" by saying: ""The Russians use the money we give th...",1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,0,0,1
1893,E08022,"Imagine someone is arguing in favor of ""We should stop buying Russian gas"" by saying: ""The cost of gas will be higher. But I...",0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1
1894,E08024,"Imagine someone is arguing in favor of ""We should strengthen our ties with Ukraine and prepare for its membership in the EU""...",0,1,0,0,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,1


In [55]:
def result2tsv(result1,valid_ds):
    validData = valid_ds
    dictvalidT =validData.to_dict("list")

    validPred = {}
    validPred["Argument ID"] = dictvalidT["Argument ID"]
    validPred["sentence"] = dictvalidT["sentence"]

    for x in label_vocab:
        validPred[x] = []
    
    for x in range(len(result1)):
        types = result1[x].split(",")
        if types == ['']:
            for y in label_vocab:
                validPred[y].append(0)
        else:  
            for z in label_vocab:
                if z in types:
                    validPred[z].append(1)
                else:
                    validPred[z].append(0)
    validData = valid_ds
    for x in label_vocab:
        for y in range(len(validData[x])):
            validData[x].iloc[y] = validPred[x][y]
    
    validData.drop(columns=["sentence"],inplace=True)
    return validData

In [56]:
valid = result2tsv(validresults,valid_df)
valid.to_csv('validRoberta.tsv',columns=valid.columns.tolist(),
            sep='\t',
            index=False)

In [57]:
validzhihu_df = argumentadd(datadir+'/arguments-validation-zhihu.tsv',datadir + '/labels-validation-zhihu.tsv')

In [58]:
validzhihu_data = TransformerDataset(validzhihu_df)

validzhihu_dataloader = DataLoader(validzhihu_data, batch_size=config.BATCH_SIZE, shuffle=False)

In [61]:
validzhihu_pred = predict(best_model,validzhihu_dataloader)
validzhihu_probs = F.sigmoid(torch.tensor(validzhihu_pred))
validzhihuresults = computeResult(validzhihu_probs)

In [62]:
validzhihu = result2tsv(validzhihuresults,validzhihu_df)
validzhihu.to_csv('validzhihuRoberta.tsv',columns=validzhihu.columns.tolist(),
            sep='\t',
            index=False)